# Space

In [ ]:
import sys
import os 
import logging
import pandas as pd
from pprint import pprint 
from IPython.display import display, HTML

KEY = '1-WORKSPACE'
WORKSPACE_PATH = os.getcwd().split(KEY)[0]
print(WORKSPACE_PATH); os.chdir(WORKSPACE_PATH)
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='[%(levelname)s:%(asctime)s:(%(filename)s@%(lineno)d %(name)s)]: %(message)s')

SPACE = {
    'DATA_RAW': f'_Data/0-Data_Raw',
    'DATA_RFT': f'_Data/1-Data_RFT',
    'DATA_CASE': f'_Data/2-Data_CASE',
    'DATA_AIDATA': f'_Data/3-Data_AIDATA',
    'DATA_EXTERNAL': f'code/external',
    'CODE_FN': f'code/pipeline', 
}
assert os.path.exists(SPACE['CODE_FN']), f'{SPACE["CODE_FN"]} not found'

print(SPACE['CODE_FN'])
sys.path.append(SPACE['CODE_FN'])

# CF Data

In [ ]:
from recfldtkn.aidata_base.entry import EntryAIData_Builder
import datasets

OneAIDataName = 'DietEventBench'
CF_DataName = 'DietEvent-CGM5MinEntry-1ea9d787eef20fb7'
CohortName_list = ['WellDoc2022CGM']
CF_DataName_list = [f'{i}/{CF_DataName}' for i in CohortName_list]


entry = EntryAIData_Builder(SPACE = SPACE)

dataset = entry.merge_one_cf_dataset(CF_DataName_list)
data_config = dataset.info.config_name 
print('total', dataset)

In [ ]:
CFName = 'HM5MinStep'

interval_delta = pd.Timedelta(minutes=5)
idx2tkn = [
    pd.Timestamp('2022-01-01 00:00:00') + interval_delta * i for i in range(24 * 12)
]
idx2tkn = [f'{i.hour:02d}:{i.minute:02d}' for i in idx2tkn]
tkn2idx = {tkn: idx for idx, tkn in enumerate(idx2tkn)}
CF_to_CFvocab = data_config['CF_to_CFvocab']
CF_to_CFvocab[CFName] = {'idx2tkn': idx2tkn, 'tkn2idx': tkn2idx, }

In [ ]:
Data = {'ds_case': dataset}

# OUTPUT 1: UniLabel

## Args

In [ ]:
OneEntryArgs = {
    # ----------------- Input Part -----------------
    'Input_Part': {
        'EntryInputMethod': '1TknInStepWt5MinHM',
        'CF_list': [
            'CGMValueBf24h',
            # 'CGMValueAf2h',
        ],
        'BeforePeriods': ['Bf24h'],
        # 'AfterPeriods': ['Af2h'],
        'TimeIndex': True, 
        'InferenceMode': False, # True, # True, # False, # True, 
        'TargetField': 'CGMValue',
        'TargetRange': [40, 400],
        # 'HM': None, 
        'HM': {'start': -24, 'unit': 'h', 'interval': '5m'},
    }, 


    # ----------------- Output Part -----------------
    'Output_Part': {
        'EntryOutputMethod': 'CausalLM',
        'selected_columns': ['input_ids', 'hm_ids', 'labels'],
        'set_transform': True,
        'num_proc': 4, 
    },
}

# Data = {'df_case': caseset.df_case, 'ds_case': caseset.ds_case
EntryOutputMethod = OneEntryArgs['Output_Part']['EntryOutputMethod']
EntryInputMethod = OneEntryArgs['Input_Part']['EntryInputMethod']

In [ ]:
from recfldtkn.aidata_base.entry import EntryAIData_Builder

entry = EntryAIData_Builder(OneEntryArgs = OneEntryArgs, 
                            SPACE = SPACE)
tfm_fn_AIInputData = entry.tfm_fn_AIInputData
entry_fn_AIInputData = entry.entry_fn_AIInputData

## Function

In [ ]:
import inspect

def get_OUTPUT_CFs(OneEntryArgs):
    if 'Output_Part' not in OneEntryArgs:
        return []
    else:
        return OneEntryArgs['Output_Part'].get('CF_list', [])
get_OUTPUT_CFs.fn_string = inspect.getsource(get_OUTPUT_CFs)

In [ ]:
import numpy as np 


def transform_fn_output(examples, tfm_fn_AIInputData, OneEntryArgs, CF_to_CFvocab):
    examples_tfm = tfm_fn_AIInputData(examples, OneEntryArgs, CF_to_CFvocab)
    # examples_tfm['labels'] = torch.LongTensor([[i] for i in examples['Labeling']])
    examples_tfm['labels'] = examples_tfm['input_ids'].clone() 
    return examples_tfm

transform_fn_output.fn_string = inspect.getsource(transform_fn_output)



def entry_fn_AITaskData(Data, 
                        CF_to_CFvocab, 
                        OneEntryArgs,
                        tfm_fn_AIInputData = None,
                        entry_fn_AIInputData = None,
                        ):

    # InputCFs = OneEntryArgs['Input_FullArgs']['INPUT_CFs_Args']['InputCFs']
    transform_fn = lambda examples: transform_fn_output(examples, tfm_fn_AIInputData, OneEntryArgs, CF_to_CFvocab)
    ds_case = Data['ds_case']

    if type(ds_case) == pd.DataFrame:
        ds_case = datasets.Dataset.from_pandas(ds_case)

    # ds_case.set_transform(transform_fn)
    # use_map = OneEntryArgs.get('use_map', False)
    Output_Part = OneEntryArgs['Output_Part']
    num_proc = Output_Part.get('num_proc', 4)
    set_transform = Output_Part.get('set_transform', True)
    if set_transform == True:
        ds_case.set_transform(transform_fn)
        ds_tfm = ds_case
    else:
        old_cols = ds_case.column_names
        if 'selected_columns' in Output_Part:
            old_cols = [i for i in old_cols if i not in Output_Part['selected_columns']]
        ds_tfm = ds_case.map(transform_fn, batched = True, num_proc = num_proc)
        ds_tfm = ds_tfm.remove_columns(old_cols)

    Data['ds_tfm'] = ds_tfm

    return Data


entry_fn_AITaskData.fn_string = inspect.getsource(entry_fn_AITaskData)  

In [ ]:
Data = entry_fn_AITaskData(Data, 
                           CF_to_CFvocab, 
                           OneEntryArgs,
                           tfm_fn_AIInputData,
                           entry_fn_AIInputData)

ds_tfm = Data['ds_tfm']
ds_tfm

In [ ]:
batch = ds_tfm[:4]
for k, v in batch.items():
    print(k, v.shape)

In [ ]:
from recfldtkn.base import Base
from recfldtkn.aidata_base.entry import AIDATA_ENTRYOUTPUT_PATH

prefix = [
    'import torch',
    'import pandas as pd', 
    'import numpy as np', 
    'import datasets',
    ]
fn_variables = [
    get_OUTPUT_CFs,
    transform_fn_output,
    entry_fn_AITaskData,
]
pycode = Base.convert_variables_to_pystirng(fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(SPACE['CODE_FN'], AIDATA_ENTRYOUTPUT_PATH, f'{EntryOutputMethod}.py')
print(pypath)
if not os.path.exists(os.path.dirname(pypath)): os.makedirs(os.path.dirname(pypath))
with open(pypath, 'w') as file: file.write(pycode)